In [4]:
import warnings
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime, timedelta
import warnings
import plotly.express as px
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
# Update the cusip list as needed https://www.pimco.com/us/en/product-finder?filters=assetClass%3Dfxi%26fixedIncome%3Dinf_ass 
fund_dict = {
    'PIMCO 1-5 Year U.S. TIPS Index Exchange-Traded Fund':'72201R205',
    'PIMCO 15+ Year U.S. TIPS Index Exchange-Traded Fund':'72201R304',
    'PIMCO Broad U.S. TIPS Index Exchange-Traded Fund':'72201R403'
}

# Update target fund and date range as needed 
target_fund = 'PIMCO 1-5 Year U.S. TIPS Index Exchange-Traded Fund'
start_date = '2024-01-01'
end_date = datetime.today().strftime('%Y-%m-%d') 

# these are from website inspection, update as needed
headers = {
    'authority': 'fund-ui.pimco.com',
    'client': 'WEB',
    'countrycode': 'US',
    'userrole': 'IND'
}

In [5]:
def get_fund_data(cusip: str, start_date: str, end_date: str, interval: int) -> pd.DataFrame:

    start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    all_data = []
    
    current_date = start_date_dt
    while current_date <= end_date_dt:
        as_of_date = current_date.strftime("%Y-%m-%d")
        url = f"https://fund-ui.pimco.com/fund-detail-api/api/funds/{cusip}/topTenHoldings/export?asOfDate={as_of_date}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200 or response.status_code == 201:
            excel_data = BytesIO(response.content)
            df = pd.read_excel(excel_data, header=1, engine="openpyxl")
            df['Date'] = as_of_date
            df['id'] = df['CUSIP'] + ' ' + df['Description']
            df['Notional/Par Value Quantity/Units'] = pd.to_numeric(df['Notional/Par Value Quantity/Units'].astype(str).str.replace(',', ''), errors='coerce')
            df = df[['Date', 'id', 'Notional/Par Value Quantity/Units']].dropna()
            all_data.append(df)
        else:
            print(f"Failed to download file for date {as_of_date}. Status code: {response.status_code}")
            print(response.text)
        
        # 7 is weekly frequency and 30 is monthly frequency etc
        current_date += timedelta(days=interval)

    if all_data:
        full_data = pd.concat(all_data)
        pivot_df = full_data.pivot_table(index='Date', columns='id', values='Notional/Par Value Quantity/Units')
        return pivot_df
    else:
        print("No data retrieved.")
        return None
    
def plot_fund_holding(fund_name, start_date, end_date, interval):
    cusip = fund_dict[fund_name]
    pivot_df = get_fund_data(cusip, start_date, end_date, interval)

    if pivot_df is not None:
        pivot_df_reset = pivot_df.reset_index()
        melted_df = pivot_df_reset.melt(id_vars=['Date'], var_name='Holding', value_name='Notional/Par Value Quantity/Units')
        fig = px.line(melted_df, x='Date', y='Notional/Par Value Quantity/Units', color='Holding',
                      labels={'Notional/Par Value Quantity/Units': 'Notional'},
                      title=f'Top Holdings Over Time for {fund_name}')
        fig.update_layout(
            legend=dict(
                title="Holdings",
                itemclick="toggle",  
                itemdoubleclick="toggleothers"  
            )
        )
        fig.for_each_trace(lambda trace: trace.update(visible="legendonly"))
        fig.show()
    else:
        print("No data to plot.")
        
    return pivot_df

In [6]:
# note no file updates on weekends
plot_fund_holding(fund_name=target_fund, start_date=start_date, end_date=end_date, interval=1)

Failed to download file for date 2024-01-01. Status code: 204

Failed to download file for date 2024-01-06. Status code: 204

Failed to download file for date 2024-01-07. Status code: 204

Failed to download file for date 2024-01-13. Status code: 204

Failed to download file for date 2024-01-14. Status code: 204

Failed to download file for date 2024-01-20. Status code: 204

Failed to download file for date 2024-01-21. Status code: 204

Failed to download file for date 2024-01-27. Status code: 204

Failed to download file for date 2024-01-28. Status code: 204

Failed to download file for date 2024-02-03. Status code: 204

Failed to download file for date 2024-02-04. Status code: 204

Failed to download file for date 2024-02-10. Status code: 204

Failed to download file for date 2024-02-11. Status code: 204

Failed to download file for date 2024-02-17. Status code: 204

Failed to download file for date 2024-02-18. Status code: 204

Failed to download file for date 2024-02-24. Status cod

id,85748R009 FIXED INC CLEARING CORP.REPO,912797HQ3 TREASURY BILL 05/24 0.00000,912797KT3 TREASURY BILL 10/24 0.00000,912810FR4 TSY INFL IX N/B 01/25 2.375,912810PV4 TSY INFL IX N/B 01/28 1.75,912810PZ5 TSY INFL IX N/B 01/29 2.5,9128282L3 TSY INFL IX N/B 07/27 0.375,9128283R9 TSY INFL IX N/B 01/28 0.5,9128285W6 TSY INFL IX N/B 01/29 0.875,9128287D6 TSY INFL IX N/B 07/29 0.25,...,91282CFR7 TSY INFL IX N/B 10/27 1.625,91282CGK1 TSY INFL IX N/B 01/33 1.125,91282CGW5 TSY INFL IX N/B 04/28 1.25,91282CJH5 TSY INFL IX N/B 10/28 2.375,91282CJY8 TSY INFL IX N/B 01/34 1.75,91282CKL4 TSY INFL IX N/B 04/29 2.125,91282CKL4 WI TREASURY SEC 04/29 2.125,91282CLE9 TSY INFL IX N/B 07/34 1.875,999USDZ92 US DOLLAR,— NET OTHER ASSETS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02,138462.78,NaN,NaN,37760046.74,25262869.72,21163046.80,19251340.80,45049913.52,26735033.52,7023476.00,...,22788624.56,NaN,16166690.43,NaN,NaN,NaN,NaN,NaN,86.92,0.0
2024-01-03,NaN,NaN,NaN,37757733.04,25261321.36,21161717.50,19249963.08,45047023.76,26733278.00,7023008.80,...,22787088.48,NaN,16165587.02,NaN,NaN,NaN,NaN,NaN,138569.71,0.0
2024-01-04,NaN,NaN,NaN,37755187.97,25259600.96,21160388.20,19248738.44,45044134.00,26731522.48,7022600.00,...,22785771.84,NaN,16164641.24,NaN,NaN,NaN,NaN,NaN,138579.72,0.0
2024-01-05,NaN,NaN,NaN,37747784.13,25254611.80,21156252.60,19245064.52,45035103.50,26726255.92,7021198.40,...,22781163.60,NaN,16161488.64,NaN,NaN,NaN,NaN,NaN,138609.75,0.0
2024-01-08,NaN,NaN,NaN,37745470.43,25253063.44,21154775.60,19243686.80,45032213.74,26724500.40,7020731.20,...,22779846.96,NaN,16160385.23,NaN,NaN,NaN,NaN,NaN,35356.45,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-15,NaN,NaN,NaN,NaN,21582507.54,18096712.83,16481680.20,38521194.03,22857807.22,5990281.56,...,19483789.60,NaN,13840323.84,4710351.15,2902167.6,21996014.04,NaN,2746936.08,558623.89,0.0
2024-08-16,NaN,NaN,NaN,NaN,21583227.24,18097331.28,16482321.95,38522403.99,22858542.18,5990476.68,...,19484340.82,NaN,13840852.48,4710534.87,2902281.2,21996663.81,NaN,2747045.84,558623.89,0.0
2024-08-19,NaN,NaN,NaN,NaN,21583371.18,18097454.97,16482450.30,38522706.48,22858725.92,5990574.24,...,19484524.56,NaN,13840984.64,4710580.80,2902309.6,21996880.40,NaN,2747073.28,558623.89,0.0
